## Задача 1

In [ ]:
Условие:

CREATE TABLE Transactions (
  Client_id INT NOT NULL,
  Report_date DATE,
  Txn_amount INT 
);


INSERT INTO Transactions
    (Client_id, Report_date, Txn_amount)
VALUES
    (123, '2017.01.01', 50000);
    

CREATE TABLE Rates (
  Report_date DATE,
  Ccy_code INT,
  CCy_rate DECIMAL(5,2) 
);

INSERT INTO Rates
VALUES
    ('2016.12.30', 840, 60.58),
    ('2017.01.09', 840, 61.01);
   

In [ ]:
Задание: Напишите sql запрос, который будет переводить сумму транзакций в usd (ccy_code = 840) 
с учетом того, что в таблице rates данные только за рабочие дни. Транзакции, совершенные в 
выходные, пересчитываются по курсу последнего рабочего дня перед праздником/выходным. 
Результат: Клиент, дата, сумма операций в usd.

Запрос:

SELECT Client_id, Report_date, Txn_amount *
    (SELECT CCy_rate
     FROM Rates
     WHERE Rates.Report_date <= Transactions.Report_date AND Rates.Ccy_code = 840 ORDER BY Rates.Report_date 
     DESC LIMIT 1) AS Txn_amount_USD

FROM Transactions;

## Задача 2.

In [ ]:
Условие:

CREATE TABLE VSP_oper_data (
  Client_id INT,
  Report_date DATE,
  VSP_Number VARCHAR(10),
  Txn_type VARCHAR(10),
  Txn_amount INT
);

INSERT INTO VSP_oper_data (Client_id, Report_date, VSP_Number, Txn_type, Txn_amount)
VALUES
    -- May 1
    (1001, '2017-05-01', '1234/0123', 'debit', 5000),
    (1002, '2017-05-01', '5678/0456', 'credit', 3000),
    -- May 2
    (1002, '2017-05-02', '5678/0456', 'debit', 2000),
    (1003, '2017-05-02', '9101/0789', 'credit', 6000),
    -- May 30
    (1001, '2017-05-30', '1234/0123', 'credit', 8000),
    (1002, '2017-05-30', '5678/0456', 'debit', 2000),
    (1003, '2017-05-30', '9101/0789', 'credit', 6000),
    -- May 31
    (1001, '2017-05-31', '1234/0123', 'debit', 5000),
    (1003, '2017-05-31', '9101/0789', 'debit', 8000),

    -- June 1
    (1001, '2017-06-01', '1234/0123', 'debit', 4000),
    (1003, '2017-06-01', '9101/0789', 'debit', 7000),
    -- June 2
    (1001, '2017-06-02', '1234/0123', 'credit', 3500),
    (1002, '2017-06-02', '5678/0456', 'debit', 1800),
    (1007, '2017-06-02', '9101/0789', 'credit', 6500),

    -- June 30
    (1001, '2017-06-30', '1234/0123', 'debit', 4500),
    (1004, '2017-06-30', '5678/0456', 'credit', 2700),
    (1005, '2017-06-30', '9101/0789', 'debit', 7200);

In [ ]:
Задание: напишите sql запрос, который для каждого клиента выводит сумму debit, credit операций 
и последнее посещенное VSP по месяцам

Запрос:

SELECT Client_id, DATE_FORMAT(Report_date, '%Y-%m') AS Report_date_,


SUM(CASE WHEN Txn_type="debit" THEN Txn_amount ELSE 0 END) AS Debit_amount, SUM(CASE WHEN Txn_type="credit" THEN Txn_amount ELSE 0 END) AS Credit_amount, 
(select VSP_Number
 from VSP_oper_data t
 where t.Client_id = main.Client_id and DATE_FORMAT(t.Report_date, '%Y-%m') = Report_date_ 
 ORDER BY Report_date DESC LIMIT 1) AS Last_VSP

FROM VSP_oper_data main
GROUP BY Client_id, DATE_FORMAT(Report_date, '%Y-%m');

In [ ]:
Задание: напишите sql запрос, который для каждого клиента выведет долю debit операций 
клиента к debit операциям всех клиентов по месяцам. 

Запрос:

SELECT Client_id, DATE_FORMAT(Report_date, '%Y-%m') AS Report_date_,
SUM(CASE WHEN Txn_type="debit" THEN Txn_amount ELSE 0 END) /
(SELECT SUM(CASE WHEN Txn_type="debit" THEN Txn_amount ELSE 0 END) FROM VSP_oper_data t WHERE DATE_FORMAT(t.Report_date, '%Y-%m') = Report_date_) AS Ratio

FROM VSP_oper_data main
GROUP BY Client_id, DATE_FORMAT(Report_date, '%Y-%m');